In [ ]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
%%appyter markdown

# KINOMEscan Data Visualization

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import textwrap 
import os.path
import urllib.request
import xlrd
from collections import defaultdict

In [ ]:
%%appyter hide_code

{% do SectionField(name ='title', title = 'Visualize KINOMEscan Data', 
                   subtitle = 'KINOMEscan assay platform data show small molecule binding inhibition of protein kinases', 
                   img = 'poll.png')%}

{% do SectionField(name = 'section0', title = 'Input a Small Molecule and/or Kinase for KINOMEscan Data', 
                   subtitle = 'Input a small molecule to visualize the kinases it binds to ' + 
                   '-AND/OR- input a kinase to visualize the small molecules that bind it.', img = 'database.png')%} 

{% do SectionField(name ='title2', title = 'Visualize Target Affinity Spectrum (TAS) Data',
                   subtitle = 'Target Affinity Spectrum (TAS) vectors summarize binding information ' +
                   'binding information from multiple assay formats.', img = 'poll.png')%}

{% do SectionField(name = 'section', title = 'Input a Small Molecule and/or Kinase for TAS Data', 
                   subtitle = 'Input a small molecule to visualize the molecules it binds to ' + 
                   '-AND/OR- input a kinase to visualize the small molecules that bind it.', img = 'database.png')%} 

{% do SectionField(name = 'section2', title = 'Upload or Enter a Gene/Protein List', 
                   subtitle = 'Upload or enter a gene/protein list (one gene per row) to visualize the ' +
                   'small molecules that bind to the protein kinases in that list.', img = 'file-upload.png')%}  


In [ ]:
%%appyter markdown

### Generate table of kinases for small molecule input based on KINOMEscan data, with either equilibrium dissociation constant Kd or % Control 
For both Kd and % control, the lower the number, the higher the binding affinity and inhibition

In [ ]:
%%appyter code_exec

# if small molecule name inputted, will go through its CSV file and sort by target affinity

sm_name_kinome = {{ StringField(
        name = 'Small molecule name',
        label = 'Small molecule name (e.g. AC220, (s)-CR8)',
        description = 'One small molecule input. Examples include: (s)-CR8, AC220, Afatinib',
        default = '',
        section = 'section0'
    )}}

if (sm_name_kinome != ''):
    kinome_scan_df = pd.read_excel('http://lincs.hms.harvard.edu/wordpress/wp-content/uploads/2013/11/HMS-LINCS_KinomeScan_Datasets_2018-01-18.xlsx')
    dataset_id = kinome_scan_df.loc[kinome_scan_df['sm_name'] == sm_name_kinome, 'dataset_id']

    # remove the row number
    dataset_id_list = []
    for set_id in dataset_id:
        dataset_id_list.append(set_id)
    
    dataset_id_string = str(dataset_id_list[0])
    sm_url = 'http://lincs.hms.harvard.edu/db/datasets/{}/results?search=&output_type=.xlsx'.format(dataset_id_string)
    sm_kinome_data = pd.read_excel(sm_url)
    sm_df = pd.DataFrame(sm_kinome_data)

    percentages = defaultdict(set)
    percentages_exists = False
    kds = defaultdict(set)

    if '% Control' in sm_df:
        percentages_exists = True
        sm_df = sm_df.sort_values(by = '% Control')
        percentages = dict(zip(sm_df['Protein Name'], sm_df['% Control']))

        # Print in table format
        print('The % control values that equal 100 are omitted due to its indication that there is no inhibition whatsoever.')
        print ("{:<49} {:<15}".format('\033[1m' + 'Protein Name','% Control' + '\033[0m'))
        for k, v in percentages.items():
            if (percentages[k] != 100.0):
                print ("{:<45} {:<15}".format(k, v))
    else:
        sm_df = sm_df[sm_df['Kd'].notna()] # remove all rows with no value
        sm_df = sm_df.sort_values(by = ['Kd'])
        kds = dict(zip(sm_df['Protein Name'], sm_df['Kd']))
        
        # Print in table format
        print ("{:<49} {:<15}".format('\033[1m' + 'Protein Name', 'Kd' + '\033[0m'))
        for k, v in kds.items():
            print ("{:<45} {:<15}".format(k, v))

else:
    print('There was no small molecule input.')

In [ ]:
%%appyter markdown

### Generate bar chart for small molecule input based on TAS vectors
Hover over bar(s) to see kinases.

In [ ]:
# Function that all bar charts will use to make sure the data labels are readable 
def prep_and_wrap(aList):
    add_commas = ', '.join(aList)
    return '<br>'.join(textwrap.wrap(add_commas, width=50))    

In [ ]:
# if there was a small molecule input, generate and display the bar chart
if (sm_name_kinome != ''): 
    if percentages_exists:
        keys_0 = []
        keys_20 = []
        keys_40 = []
        keys_60 = []
        keys_80 = []
        
        for key in percentages.keys():
            if percentages[key] < 20.0:
                keys_0.append(key)
            elif percentages[key] < 40.0:
                keys_20.append(key)
            elif percentages[key] < 60.0:
                keys_40.append(key)
            elif percentages[key] < 80.0:
                keys_60.append(key)
            else:
                keys_80.append(key)
                
        # Prepare and wrap text for labels 
        kinase0_label = prep_and_wrap(keys_0)
        kinase20_label = prep_and_wrap(keys_20)
        kinase40_label = prep_and_wrap(keys_40)
        kinase60_label = prep_and_wrap(keys_60)
        kinase80_label = prep_and_wrap(keys_80)
        
        x = ['% Control < 20%', '20% ≤ % Control < 40%', '40% ≤ % Control < 60%', '60% ≤ % Control < 80%', '80% ≤ % Control < 100%']
        y = [len(keys_0), len(keys_20), len(keys_40), len(keys_60), len(keys_80)]

        fig = go.Figure(data=[go.Bar(x = x, y = y, text = y, textposition = 'auto',
                                 hovertext = [kinase0_label, kinase20_label, kinase40_label, kinase60_label, kinase80_label], 
                                 hoverlabel = dict(font = dict(size = 18)),
                                 marker = {'color': y,
                                            'colorscale': 'Plotly3'})])

        fig.update_layout(
            title = {
                'text': 'Kinases bound by ' + sm_name_kinome,
                'y':0.87,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top',
            },
            xaxis_title = "% Control",
            yaxis_title = "Kinases",
            font = dict(
                #family = "Courier New, monospace",
                size = 18,
                color = 'black'
            )
        )

        fig.show()

    else:
        kds_1 = []
        kds_2 = []
        kds_3 = []
        
        for key in kds.keys():
            if kds[key] < 100.0:
                kds_1.append(key)
            elif kds[key] < 1000.0:
                kds_2.append(key)
            elif kds[key] < 10000.0:
                kds_3.append(key)
                
        # Prepare and wrap text for labels 
        kds1_label = prep_and_wrap(kds_1)
        kds2_label = prep_and_wrap(kds_2)
        kds3_label = prep_and_wrap(kds_3)
        
        x = ['Kd < 100 nM', '100 nM ≤ Kd < 1µM', '1µM ≤ Kd < 10 µM']
        y = [len(kds_1), len(kds_2), len(kds_3)]

        fig = go.Figure(data=[go.Bar(x = x, y = y, text = y, textposition = 'auto',
                                 hovertext = [kds1_label, kds2_label, kds3_label], 
                                 hoverlabel = dict(font = dict(size = 18)),
                                 marker = {'color': y,
                                            'colorscale': 'Plotly3'})])

        fig.update_layout(
            title = {
                'text': 'Kinases bound by ' + sm_name_kinome,
                'y':0.87,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top',
            },
            xaxis_title = "Kd",
            yaxis_title = "Kinases",
            font = dict(
                #family = "Courier New, monospace",
                size = 18,
                color = 'black'
            )
        )
        
        fig.show()
else:
    print('There was no small molecule input.')

In [ ]:
%%appyter markdown

### Generate list of kinases for small molecule input based on TAS vectors

In [ ]:
%%appyter code_exec

# if small molecule name inputted, will go through its CSV file and sort by target affinity

sm_name = {{ StringField(
        name = 'Small molecule name',
        label = 'Small molecule name (e.g. AC220)',
        description = 'One small molecule input. Examples include: (s)-CR8, AC220, Afatinib',
        default = '',
        section = 'section'
    )}}

kinase1_string = ''
kinase2_string = ''
kinase3_string = ''

if (sm_name != ''):
    kinome_df = pd.read_excel('http://lincs.hms.harvard.edu/wordpress/wp-content/uploads/2013/11/HMS-LINCS_KinomeScan_Datasets_2018-01-18.xlsx')
    kinome_df['sm_hms_id'] = kinome_df['sm_hms_id'].str.replace(r'\D', '') #remove HMSL before the ID
    hms_id = kinome_df.loc[kinome_df['sm_name'] == sm_name, 'sm_hms_id']

    # remove the row number
    hms_id_list = []
    for hms_id in hms_id:
        hms_id_list.append(hms_id)
    
    hms_id_string = str(hms_id_list[0])
    old_url = 'http://lincs.hms.harvard.edu/db/datasets/20000/results?small+molecules=HMS_ID&output_type=.csv'
    new_url = old_url.replace("HMS_ID", hms_id_string) # fills in HMS_ID with the correct small molecule 
    data = pd.read_csv(new_url)
    df = pd.DataFrame(data)

    target_affinity_1 = df.loc[df['Binding Class'] == 1, 'HUGO Gene Symbol']
    target_affinity_2 = df.loc[df['Binding Class'] == 2, 'HUGO Gene Symbol']
    target_affinity_3 = df.loc[df['Binding Class'] == 3, 'HUGO Gene Symbol']
    target_affinity_10 = df.loc[df['Binding Class'] == 10, 'HUGO Gene Symbol'] 

    kinase1_list = []
    kinase2_list = []
    kinase3_list = []

    for kinase in target_affinity_1:
        if kinase not in kinase1_list: # removes duplicates
            kinase1_list.append(str(kinase))

    if len(kinase1_list) != 0:
        print(sm_name + ' binds to the following kinases with Kd < 100 nM: ')
        kinase1_string = ', '.join(kinase1_list)
        print(kinase1_string)
        print()

    for kinase in target_affinity_2:
        if kinase not in kinase2_list: # removes duplicates
            kinase2_list.append(str(kinase))

    if len(kinase2_list) != 0:
        print(sm_name + ' binds to the following kinases with 100 nM ≤ Kd < 1µM: ')
        kinase2_string = ', '.join(kinase2_list)
        print(kinase2_string)
        print()

    for kinase in target_affinity_3:
        if kinase not in kinase3_list: # removes duplicates
            kinase3_list.append(str(kinase))

    if len(kinase3_list) != 0:
        print(sm_name + " binds to the following kinases with 1µM ≤ Kd < 10 µM: ")
        kinase3_string = ', '.join(kinase3_list)
        print(kinase3_string)

else:
    print('There was no small molecule input.')

In [ ]:
%%appyter markdown

### Generate bar chart for small molecule input based on TAS vectors
Hover over bar(s) to see kinases.

In [ ]:
# if there was a small molecule input, generate and display the bar chart
if (sm_name != ''): 
    x = ['Kd < 100 nM', '100 nM ≤ Kd < 1µM', '1µM ≤ Kd < 10 µM']
    y = [len(kinase1_list), len(kinase2_list), len(kinase3_list)]

    # Text wraparound
    kinase1_lab = '<br>'.join(textwrap.wrap(kinase1_string, width=50))
    kinase2_lab ='<br>'.join(textwrap.wrap(kinase2_string, width=50))
    kinase3_lab ='<br>'.join(textwrap.wrap(kinase3_string, width=50))

    fig = go.Figure(data=[go.Bar(x = x, y = y, text = y, textposition = 'auto',
                                 hovertext = [kinase1_lab, kinase2_lab, kinase3_lab], 
                                 hoverlabel = dict(font = dict(size = 18)),
                                 marker = {'color': y,
                                            'colorscale': 'Plotly3'})])

    fig.update_layout(
        title = {
            'text': 'Kinases bound by ' + sm_name,
            'y':0.87,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
        },
        xaxis_title = "Equilibrium Dissociation Constant",
        yaxis_title = "Kinases",
        font = dict(
            #family = "Courier New, monospace",
            size = 18,
            color = 'black'
        )
    )

    fig.show()

else:
    print('There was no small molecule input.')

In [ ]:
%%appyter markdown

### Generate list of small molecules for kinase input based on TAS vectors
Includes download of the KINOMEscan GMT files. For more information on how this was assembled, go to 
the KinomeScan-Appyter folder in the HarmonizomePythonScripts repository
(https://github.com/MaayanLab/HarmonizomePythonScripts). 

In [ ]:
%%appyter code_exec

# if kinase inputted, return small molecules  
kinase_name = {{ StringField(
        name = 'Kinase name',
        label = 'Kinase name (e.g. MAP2K1)',
        description = 'One kinase input. Examples include: ABL1, ABL2, ALK, CCNA1',
        default = '',
        section = 'section'
    )}}

sm1_string = ''
sm2_string = ''
sm3_string = ''

if not os.path.exists('level1.gmt'):
    urllib.request.urlretrieve('https://gist.githubusercontent.com/serena-zhang/a5344564a9beed30e7b5a626da1c0deb/raw/2ce9fad8256de9c288cedea24c64671c0f69e9e0/level1.gmt', 'level1.gmt')
level_1 = {split_line[0]: split_line[2:] for split_line in map(lambda s: s.split('\t'), open('level1.gmt', 'r'))}
if not os.path.exists('level2.gmt'):
    urllib.request.urlretrieve('https://gist.githubusercontent.com/serena-zhang/a5344564a9beed30e7b5a626da1c0deb/raw/2ce9fad8256de9c288cedea24c64671c0f69e9e0/level2.gmt', 'level2.gmt')
level_2 = {split_line[0]: split_line[2:] for split_line in map(lambda s: s.split('\t'), open('level2.gmt', 'r'))}
if not os.path.exists('level3.gmt'):
    urllib.request.urlretrieve('https://gist.githubusercontent.com/serena-zhang/a5344564a9beed30e7b5a626da1c0deb/raw/2ce9fad8256de9c288cedea24c64671c0f69e9e0/level3.gmt', 'level3.gmt')
level_3 = {split_line[0]: split_line[2:] for split_line in map(lambda s: s.split('\t'), open('level3.gmt', 'r'))}

if (kinase_name != ''):
    # lists for each target affinity for the small molecules
    sm1_list = []
    sm2_list = []
    sm3_list = []

    if kinase_name in level_1:
        sm1_list = level_1[kinase_name]
    if kinase_name in level_2:
        sm2_list = level_2[kinase_name]
    if kinase_name in level_3:
        sm3_list = level_3[kinase_name]
        
    if len(sm1_list) != 0:
        print('Small molecules that bind to ' + kinase_name + ' with Kd < 100 nM: ')
        sm1_string = ', '.join(sm1_list)
        print(sm1_string)
        print()
    
    if len(sm2_list) != 0:
        print('Small molecules that bind to ' + kinase_name + ' with 100 nM ≤ Kd < 1µM: ')
        sm2_string = ', '.join(sm2_list)
        print(sm2_string)    
        print()

    if len(sm3_list) != 0:
        print('Small molecules that bind to ' + kinase_name + ' with 1µM ≤ Kd < 10 µM: ')
        sm3_string = ', '.join(sm3_list)
        print(sm3_string)    

else:
    print('There was no kinase input.')

In [ ]:
%%appyter markdown

### Generate bar chart for kinase input based on TAS vectors
Hover over bar(s) to see small molecules.

In [ ]:
# if there was a kinase input, generate and display the bar chart
if (kinase_name != ''):
    x = ['Kd < 100 nM', '100 nM ≤ Kd < 1µM', '1µM ≤ Kd < 10 µM']
    y = [len(sm1_list), len(sm2_list), len(sm3_list)]

    # Text wraparound
    sm1_lab = '<br>'.join(textwrap.wrap(sm1_string, width = 50))
    sm2_lab ='<br>'.join(textwrap.wrap(sm2_string, width = 50))
    sm3_lab ='<br>'.join(textwrap.wrap(sm3_string, width = 50))

    fig = go.Figure(data=[go.Bar(x = x, y = y, text = y, textposition = 'auto',
                                 hovertext = [sm1_lab, sm2_lab, sm3_lab], 
                                 hoverlabel = dict(font = dict(size = 18)),
                                 marker = {'color': y,
                                            'colorscale': 'Plotly3'})])

    fig.update_layout(
        title = {
            'text': 'Small molecules that bind ' + kinase_name,
            'y':0.87,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
        },
        xaxis_title = "Equilibrium Dissociation Constant",
        yaxis_title = "Number of small molecules",
        font = dict(
            #family = "Courier New, monospace",
            size = 18,
            color = 'black'
        )
    )
    
    fig.show()

else:
    print('There was no kinase input.')

In [ ]:
%%appyter code_exec

# Import gene list as file or from text box file
# Will choose file upload over textbox if a file is given 

gene_list_file = {{ FileField(
        name = 'Upload gene/protein list',
        label = 'Upload gene/protein list',
        description = 'Gene/protein list upload',
        default = '',
        section = 'section2'
    )}}

gene_list_input = {{ TextField(
        name = 'Input gene/protein list',
        label = 'Gene/protein list (e.g. AAK1 \n ABL1)',
        description = 'Gene/protein list input',
        default = 'AAK1 \nABL1 \nABL2 \nACVR1',
        section = 'section2',
    )}}

input_exists = False 

if gene_list_file != '':
    open_gene_list_file = open(gene_list_filename,'r')
    lines = open_gene_list_file.readlines()
    genes = [x.strip() for x in lines]
    open_gene_list_file.close()
    input_exists = True 
elif gene_list_input != '':
    genes = gene_list_input.split('\n')
    genes = [x.strip() for x in genes]
    input_exists = True 
else:
    print('No gene list was inputted or uploaded.')

if input_exists:
    # remove any non-protein kinases from the inputted list
    prot_kinase_input = set()
    for gene in genes:
        if ((gene in level_1) or (gene in level_2) or (gene in level_3)): # if gene is a kinase 
            prot_kinase_input.add(gene)

    # all the other kinases that were not inputted
    other_kinases = set() 
    for kinase in level_1.keys():
        if kinase not in prot_kinase_input:
            other_kinases.add(kinase)
    for kinase in level_2.keys():
        if kinase not in prot_kinase_input:
            other_kinases.add(kinase)
    for kinase in level_3.keys():
        if kinase not in prot_kinase_input:
            other_kinases.add(kinase)
        
    # go through prot_kinases and retrieve the small molecules that bound to at least one of them
    sm_list = []
    for kinase in prot_kinases:
        sm_list.append(level_1[kinase])
        sm_list.append(level_2[kinase])
        sm_list.append(level_3[kinase])
    
    sm_list = [item for items in sm_list for item in items] # remove nested lists 
    sm_list = [sm.strip('\n') for sm in sm_list]
    sm_list = list(set(sm_list)) # remove duplicates 
         

# want the small molecules 
    
    print(sm_list)
